In [5]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('../src/')
from harp.bonsai import load_bonsai_config
load_bonsai_config(r"C:\git\AllenNeuralDynamics\aind-vr-foraging\Bonsai")
from harp.io import read_harp_bin, load_harp_messages_from_bin

In [95]:
import clr
clr.AddReference("Bonsai.Harp")
clr.AddReference("Harp.Behavior")
import System as SystemCs
import Bonsai.Harp as harpcore
import Harp.Behavior as harpbehavior

In [134]:
def get_payload_method(register: int):
    _type = harpbehavior.Device.RegisterMap[register]
    GetPayloadMethod = _type.GetMethod("GetTimestampedPayload")
    l = lambda msg : GetPayloadMethod.Invoke(None, [harpcore.HarpMessage(msg)])
    return l

def bytes_to_harp_message(row):
    return lambda x : harpcore.HarpMessage(row.tobytes())

def array_to_harp_messages(data: np.array) -> np.array:
    return np.apply_along_axis(bytes_to_harp_message, axis=1, arr=data)

def array_to_payloads(data: np.array, address = None) -> np.array:
    if address is None:
        address = data[0].Address
    return data.apply(get_payload_method(address), axis=1)

def file_to_payloads(filename: str, address = None) -> np.array:
    data = load_harp_messages_from_bin(filename)
    arr = array_to_harp_messages(data)
    return arr


In [135]:
root = r"Z:\scratch\vr-foraging\672107\20230920T125148"
device = "behavior"
register = 32
filepath = f"{root}//{device}//register__{register}.bin"
data = read_harp_bin(filepath)
a = file_to_payloads(filepath)
a

TypeError: _bytes_to_harp_message() takes 0 positional arguments but 1 was given

In [121]:
root = r"Z:\scratch\vr-foraging\672107\20230920T125148"
device = "behavior"
register = 32
filepath = f"{root}//{device}//register__{register}.bin"
data = read_harp_bin(filepath)

msg = load_harp_messages_from_bin(filepath)
onemessage = msg[1].data.tobytes()
GetPayloadMethod = get_payload_method(register)
payload = GetPayloadMethod(onemessage)
print(payload.Seconds)
print(payload.Value)
print(payload.Value.HasFlag(harpbehavior.DigitalInputs.DIPort0))


97732.489632
DIPort0, DIPort2
True


In [123]:
register = 44
filepath = f"{root}//{device}//register__{register}.bin"
data = read_harp_bin(filepath)

msg = load_harp_messages_from_bin(filepath)
onemessage = msg[1].data.tobytes()
GetPayloadMethod = get_payload_method(register)
payload = GetPayloadMethod(onemessage)
print(payload.Seconds)
print(payload.Value.Encoder)


97725.061984
-5
